<a href="https://colab.research.google.com/github/ekrombouts/GenCareAI/blob/work_in_progress/scripts/work_in_progress/405_SAMPC_long.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets

from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd


In [ ]:
seed = 6
# Path for pushing dataset to HuggingFace Hub
path_hf_sampc_long = "ekrombouts/Galaxy_SAMPC_long"
# Commit message for the push
commit_message = "Galaxy_SAMPC dataset to long format"

In [ ]:
# Load the dataset from HuggingFace
dataset = load_dataset("ekrombouts/Galaxy_SAMPC")

# Combine train, test, and validation splits into a single DataFrame
df = pd.concat([dataset['train'].to_pandas(),
                dataset['test'].to_pandas(),
                dataset['validation'].to_pandas()],
                ignore_index=True)

In [ ]:
# Function to generate prompts for different categories
def generate_prompts(df):
    rows = []
    for idx, row in df.iterrows():
        notes = row['notes']
        prompts = {
            'somatiek': f"Lees de volgende rapportages en beschrijf de lichamelijke klachten van de cliënt.\n\nRapportages:\n{notes}\n\nGeef de output als lijst van strings: ['foo', 'bar'].\n\nBeschrijf de lichamelijke klachten:",
            'adl': f"Lees de volgende rapportages en beschrijf welke hulp de cliënt nodig heeft bij wassen en kleden.\n\nRapportages:\n{notes}\n\nBeschrijf de hulp bij wassen en kleden:",
            'mobiliteit': f"Lees de volgende rapportages en beschrijf de mobiliteit van de cliënt.\n\nRapportages:\n{notes}\n\nBeschrijf de mobiliteit van de cliënt:",
            'continentie': f"Lees de volgende rapportages en beschrijf de continentie van de cliënt.\n\nRapportages:\n{notes}\n\nBeschrijf de continentie van de cliënt:",
            'maatschappelijk': f"Lees de volgende rapportages en beschrijf bijzonderheden rondom familie en dagbesteding van de cliënt.\n\nRapportages:\n{notes}\n\nBeschrijf de familie en dagbesteding:",
            'psychisch': f"Lees de volgende rapportages en beschrijf de cognitie en gedragsproblemen van de cliënt.\n\nRapportages:\n{notes}\n\nGeef de output als lijst van strings: ['foo', 'bar'].\n\nBeschrijf cognitie en gedragsproblemen:"
        }

        for category, prompt in prompts.items():
            rows.append({'notes': row['notes'], 'prompt': prompt, 'reference': row[category], 'category': category})

    return pd.DataFrame(rows)

In [ ]:
# Generate prompts and create a new DataFrame
df_prompts = generate_prompts(df)

# Convert the 'reference' column to string type
df_prompts['reference'] = df_prompts['reference'].astype(str)

In [ ]:
# Convert the DataFrame to a HuggingFace Dataset object
dataset = Dataset.from_pandas(df_prompts)

# Split the dataset into training and test/validation splits
train_testvalid_split = dataset.train_test_split(test_size=0.2, seed=seed)

# Further split the test set into validation and test sets
test_valid_split = train_testvalid_split['test'].train_test_split(test_size=0.5, seed=seed)

# Create a DatasetDict object to hold the splits
dataset_dict = DatasetDict({
    'train': train_testvalid_split['train'],
    'validation': test_valid_split['train'],
    'test': test_valid_split['test'],
})

# Push the dataset to HuggingFace Hub with the specified path and commit message
dataset_dict.push_to_hub(path_hf_sampc_long,
                         commit_message=commit_message,
                         private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ekrombouts/Galaxy_SAMPC_long/commit/fd86220752055aacf1313bb6f62508254a3c0a54', commit_message='Galaxy_SAMPC dataset to long format', commit_description='', oid='fd86220752055aacf1313bb6f62508254a3c0a54', pr_url=None, pr_revision=None, pr_num=None)